In [2]:
from sklearn.model_selection import train_test_split,cross_val_score,KFold
from sklearn.feature_selection import SelectFromModel
import pandas as pd
import matplotlib.pyplot as plt
from xgboost import XGBRegressor as XGBR

/Users/caroline/opt/anaconda3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [3]:
%%time
train_data = pd.read_parquet('/Users/caroline/Downloads/train_data.parquet')

CPU times: user 7.01 s, sys: 1.64 s, total: 8.66 s
Wall time: 12.5 s


In [4]:
%%time
test_data = pd.read_parquet('/Users/caroline/Downloads/test_data.parquet')
test_data.columns

CPU times: user 9.62 s, sys: 2.98 s, total: 12.6 s
Wall time: 28.8 s


Index(['customer_ID', 'S_2', 'P_2', 'D_39', 'B_1', 'B_2', 'R_1', 'S_3', 'D_41',
       'B_3',
       ...
       'D_136', 'D_137', 'D_138', 'D_139', 'D_140', 'D_141', 'D_142', 'D_143',
       'D_144', 'D_145'],
      dtype='object', length=190)

In [5]:
train_data = train_data.groupby('customer_ID').tail(2)
train_data_1 = train_data.groupby('customer_ID').tail(1)
# train_data_1.S_2= 1
train_data_2 = train_data.groupby('customer_ID').tail(-1)
# train_data_2.S_2= 0
train_data = pd.merge(train_data_1,train_data_2,how='left',on='customer_ID')
train_data.head()

,customer_ID,S_2_x,P_2_x,D_39_x,B_1_x,B_2_x,R_1_x,S_3_x,D_41_x,B_3_x,...,D_137_y,D_138_y,D_139_y,D_140_y,D_141_y,D_142_y,D_143_y,D_144_y,D_145_y,target_y
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2018-03-13,0.934745,0.009119,0.009382,1.007647,0.006104,0.135021,0.001604,0.007174,...,NaN,NaN,0.007186,0.004234,0.005086,NaN,0.005810,0.002970,0.008533,0.0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,2018-03-25,0.880519,0.178126,0.034684,1.004028,0.006911,0.165509,0.005552,0.005068,...,NaN,NaN,0.002980,0.007479,0.007870,NaN,0.003284,0.003169,0.008514,0.0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,2018-03-12,0.880875,0.009704,0.004284,0.812649,0.006450,NaN,0.003796,0.007196,...,NaN,NaN,0.007383,0.006623,0.000964,NaN,0.002202,0.000834,0.003444,0.0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,2018-03-29,0.621776,0.001083,0.012564,1.006183,0.007829,0.287766,0.004532,0.009937,...,NaN,NaN,0.002704,0.006184,0.001899,NaN,0.008183,0.005560,0.002983,0.0
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,2018-03-30,0.871900,0.005573,0.007679,0.815746,0.001247,NaN,0.000231,0.005528,...,NaN,NaN,0.002974,0.004162,0.005764,NaN,0.008154,0.006944,0.000905,0.0


In [6]:
train_traget = train_data.target_x
train_data = train_data.drop(columns=['S_2_x','S_2_y','target_x','target_y'])
train_data['target'] = train_traget

In [7]:
test_data = test_data.groupby('customer_ID').tail(2)
test_data_1 = test_data.groupby('customer_ID').tail(1)
test_data_2 = test_data.groupby('customer_ID').tail(-1)
test_data = pd.merge(test_data_1,test_data_2,how='left',on='customer_ID')
test_data.head()

,customer_ID,S_2_x,P_2_x,D_39_x,B_1_x,B_2_x,R_1_x,S_3_x,D_41_x,B_3_x,...,D_136_y,D_137_y,D_138_y,D_139_y,D_140_y,D_141_y,D_142_y,D_143_y,D_144_y,D_145_y
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,2019-10-12,0.568930,0.121385,0.010779,1.009347,0.006923,0.149413,0.000396,0.003576,...,NaN,NaN,NaN,0.005912,0.001250,0.006543,NaN,0.009160,0.003690,0.003219
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,2019-04-15,0.841177,0.126475,0.016562,1.009245,0.009715,0.112195,0.006192,0.011386,...,NaN,NaN,NaN,0.004344,0.000866,0.009120,NaN,0.002196,0.000247,0.007780
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,2019-10-16,0.697522,0.002724,0.001484,0.810072,0.002620,0.166165,0.004888,0.015938,...,NaN,NaN,NaN,1.001246,0.008894,0.896224,0.150203,1.009391,0.457819,0.092041
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,2019-04-22,0.513186,0.324828,0.149511,0.205678,0.002278,0.181200,0.005813,0.498516,...,NaN,NaN,NaN,1.008246,0.003753,0.919774,0.255263,1.008101,0.500924,0.183020
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,2019-10-22,0.254478,0.768016,0.563603,0.038021,0.503154,0.168317,0.009480,0.830857,...,NaN,NaN,NaN,0.006623,0.001140,0.009527,NaN,0.009408,0.001558,0.000525


In [8]:
test_data = test_data.drop(columns=['S_2_x','S_2_y'])

In [9]:
train_data.shape

(458913, 378)

In [10]:
test_data.shape

(924621, 377)

In [ ]:
# train_data = train_data.drop(columns=['S_2'])

In [11]:
#缺失值大于90%的特征舍弃
train_data = train_data.dropna(axis=1, thresh=int(0.90*len(train_data)))

In [12]:
train_data.shape

(458913, 300)

In [13]:
features = list(train_data.columns)
features.remove('target')

In [14]:
test_features = list(test_data.columns)
drop_features = list(set(test_features).difference(set(features)))

In [15]:
test_data = test_data.drop(columns = drop_features)

In [16]:
test_data.shape

(924621, 299)

In [17]:
train_data.columns

Index(['customer_ID', 'P_2_x', 'D_39_x', 'B_1_x', 'B_2_x', 'R_1_x', 'D_41_x',
       'B_3_x', 'D_44_x', 'B_4_x',
       ...
       'D_131_y', 'D_133_y', 'R_28_y', 'D_139_y', 'D_140_y', 'D_141_y',
       'D_143_y', 'D_144_y', 'D_145_y', 'target'],
      dtype='object', length=300)

In [18]:
test_data.columns

Index(['customer_ID', 'P_2_x', 'D_39_x', 'B_1_x', 'B_2_x', 'R_1_x', 'D_41_x',
       'B_3_x', 'D_44_x', 'B_4_x',
       ...
       'D_130_y', 'D_131_y', 'D_133_y', 'R_28_y', 'D_139_y', 'D_140_y',
       'D_141_y', 'D_143_y', 'D_144_y', 'D_145_y'],
      dtype='object', length=299)

In [19]:
import pandas as pd
df = pd.concat([train_data,test_data])
df.shape

(1383534, 300)

In [20]:
pd.set_option('display.max_row', None)
df.isnull().sum(axis=0)

customer_ID         0
P_2_x            7753
D_39_x              0
B_1_x               0
B_2_x              74
R_1_x               0
D_41_x             74
B_3_x              74
D_44_x          66907
B_4_x               0
D_45_x             76
B_5_x               0
R_2_x               0
D_47_x              0
B_6_x             178
B_7_x               0
B_8_x           16475
D_51_x              0
B_9_x               0
R_3_x               0
D_52_x           3449
P_3_x           52680
B_10_x              0
S_5_x               0
B_11_x              0
S_6_x               0
D_54_x             74
R_4_x               0
B_12_x              0
S_8_x               0
D_55_x          90815
B_13_x           3573
R_5_x               0
D_58_x              0
B_14_x              0
D_59_x          13077
D_60_x              0
B_15_x           2021
S_11_x              0
D_63_x              0
D_64_x          34832
D_65_x              0
B_16_x             74
B_18_x              0
B_19_x             74
B_20_x    

In [21]:
df = df.set_index('customer_ID')

In [22]:
X = pd.get_dummies(df.iloc[:,:-1])

In [23]:
X.columns

Index(['P_2_x', 'D_39_x', 'B_1_x', 'B_2_x', 'R_1_x', 'D_41_x', 'B_3_x',
       'D_44_x', 'B_4_x', 'D_45_x',
       ...
       'D_64_x_U', 'D_63_y_CL', 'D_63_y_CO', 'D_63_y_CR', 'D_63_y_XL',
       'D_63_y_XM', 'D_63_y_XZ', 'D_64_y_O', 'D_64_y_R', 'D_64_y_U'],
      dtype='object', length=312)

In [24]:
X_predict = X.iloc[458913:,:]
X_predict = X_predict.ffill().bfill()

In [25]:
pd.set_option('display.max_row', None)
X_predict.isnull().sum(axis=0)
X_predict.shape

(924621, 312)

In [26]:
X_train_or_test = X.iloc[:458913,:]
X_train_or_test = X_train_or_test.ffill().bfill()

In [27]:
X_train_or_test.shape

(458913, 312)

In [30]:
Y = df.iloc[:458913,-1:].values.ravel()
len(Y)

array([0., 0., 0., ..., 0., 1., 0.])

In [31]:
X_train,X_test, y_train, y_test =train_test_split(X_train_or_test,Y,test_size=0.3, random_state=123)

In [32]:
def amex_metric(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:

    def top_four_percent_captured(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
        
    def weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    def normalized_weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        y_true_pred = y_true.rename(columns={'target': 'prediction'})
        return weighted_gini(y_true, y_pred) / weighted_gini(y_true, y_true_pred)

    g = normalized_weighted_gini(y_true, y_pred)
    d = top_four_percent_captured(y_true, y_pred)

    return 0.5 * (g + d)

In [45]:
params = {
            'n_estimators':70,
            'max_depth':5
}

In [46]:
reg = XGBR(**params).fit(X_train,y_train)
test_predict = reg.predict(X_test) 
y_test1 = pd.DataFrame(y_test)
y_test1.columns = ['target']
test_predict1 = pd.DataFrame(test_predict)
test_predict1.columns = ['prediction']
score = amex_metric(y_test1,test_predict1)
score

/Users/caroline/opt/anaconda3/lib/python3.9/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


0.7660987889243318

In [ ]:
# score = []
# radom_estimators=[50,100,200,300,400,500]
# for i in radom_estimators:
#     reg = XGBR(n_estimators=i).fit(X_train,y_train)
#     test_predict = reg.predict(X_test) 
#     y_test1 = pd.DataFrame(y_test)
#     y_test1.columns = ['target']
#     test_predict1 = pd.DataFrame(test_predict)
#     test_predict1.columns = ['prediction']
#     s = amex_metric(y_test1,test_predict1)
#     print("n_estimators:{},score:{}".format(i,s))
#     score.append(s)    

In [47]:
prediction = reg.predict(X_predict)
# X_train.shape

In [48]:
prediction.shape

(924621,)

In [49]:
submission = pd.DataFrame({"customer_ID":X_predict.index,"prediction":prediction})

In [50]:
submission.head()

,customer_ID,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,-0.067681
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.010330
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.036642
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.289223
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.809930


In [56]:
submission.loc[submission['prediction']<0,'prediction'] = 0

In [57]:
submission.to_csv('submission.csv', index=False)